In [1]:
import pandas as pd
from imsciences import *

ims = dataprocessing()

path = ims.get_wd_levels(0)
path

'/Users/cameronroberts/Finances/dividends'

In [2]:
df = ims.read_and_concatenate_files(path, 'csv')

In [3]:
df

,Action,Time,ISIN,Ticker,Name,No. of shares,Price / share,Currency (Price / share),Exchange rate,Total,Currency (Total),Withholding tax,Currency (Withholding tax)
0,Dividend (Dividend),2025-02-19 12:00:34,NL0010273215,ASML,ASML,3.50,1.29,EUR,Not available,3.75,GBP,0.80,EUR
1,Dividend (Dividend),2025-02-19 12:05:37,FR0000052292,RMS,Hermes International,0.25,2.63,EUR,Not available,0.54,GBP,0.22,EUR
2,Dividend (Dividend),2024-02-29 13:48:43,US55354G1004,MSCI,MSCI,0.35,1.36,USD,Not available,0.38,GBP,0.08,USD
3,Dividend (Dividend),2024-03-27 12:43:51,US67066G1040,NVDA,Nvidia,0.20,0.03,USD,Not available,0.01,GBP,0.00,USD
4,Dividend (Dividend),2025-01-16 11:30:41,US3024913036,FMC,FMC,25.00,0.49,USD,Not available,10.09,GBP,2.18,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,Dividend (Dividend),2025-02-07 15:46:48,IE0003UVYC20,JEGP,JPMorgan Global Equity Premium Income Active (...,50.00,12.09,GBX,Not available,6.05,GBP,0.00,GBX
334,Dividend (Dividend),2025-03-07 10:28:38,IE0003UVYC20,JEGP,JPMorgan Global Equity Premium Income Active (...,50.00,11.63,GBX,Not available,5.81,GBP,-0.00,GBX
335,Dividend (Dividend),2025-02-14 12:18:57,GB0003718474,GAW,Games Workshop,7.00,80.00,GBX,Not available,5.60,GBP,0.00,GBX
336,Dividend (Dividend),2025-01-31 12:11:05,IE00BYXVWC37,STHS,PIMCO US Short-Term High Yield Corporate Bond ...,115.00,0.05,GBP,Not available,5.84,GBP,0.00,GBP


In [4]:
filename = "dividends_up_to_march_2025.csv"
df.to_csv(filename, index=False)